In [3]:
import requests
import time

weather_api_key = "4ca1ef65af4a2d76c9c1de6ea6a4b895"

def get_weather_today(city):
    url = f'http://api.openweathermap.org/data/2.5/weather?q={city}&appid=4ca1ef65af4a2d76c9c1de6ea6a4b895&units=metric&lang=kr'
    response = requests.get(url)
    data = response.json()
    lat = data['coord']['lat']
    lon = data['coord']['lon']
    return lat, lon, data

def get_weather_for_date(lat, lon, date):
    url = f'https://api.openweathermap.org/data/2.5/forecast/daily?dt={date}&lat={lat}&lon={lon}&appid=4ca1ef65af4a2d76c9c1de6ea6a4b895&units=metric&lang=kr'
    response = requests.get(url)
    return response.json()

def get_weather_today_onecall(lat, lon):
    url = f'https://api.openweathermap.org/data/2.5/forecast/daily?lat={lat}&lon={lon}&appid=4ca1ef65af4a2d76c9c1de6ea6a4b895&units=metric&lang=kr'
    response = requests.get(url)
    return response.json()

def weather_info(city):
    current_time = int(time.time())
    yesterday_time = current_time - 86400
    
    lat, lon, today_weather_city = get_weather_today(city)
    today_weather = get_weather_today_onecall(lat, lon)
    yesterday_weather = get_weather_for_date(lat, lon, yesterday_time)
    
    today_temp_max = today_weather['main']['temp_max']
    today_temp_min = today_weather['main']['temp_min']
    today_weather = today_weather['weather'][0]['main']
    
    yesterday_temp_max = yesterday_weather['main']['temp_max']
    yesterday_temp_min = yesterday_weather['main']['temp_min']

    temp_max_difference = today_temp_max - yesterday_temp_max
    temp_min_difference = today_temp_min - yesterday_temp_min

    temp_difference = today_temp_max - today_temp_min

    result = {
        "today_weather": today_weather,
        "today_temp_min": today_temp_min,
        "today_temp_max": today_temp_max,
        "temp_max_difference": temp_max_difference,
        "temp_min_difference": temp_min_difference,
        "temp_difference": temp_difference
    }

    return result

print(weather_info("Seoul"))

{'today_weather': 'Rain', 'today_temp_min': 22.69, 'today_temp_max': 23.78, 'temp_max_difference': 0.0, 'temp_min_difference': 0.0, 'temp_difference': 1.0899999999999999}


In [3]:
import openai
import os

# OpenAI API를 사용하기 위한 클라이언트 객체를 생성합니다.
openai.api_key = "" # API 키!

ASSISTANT_ID = 'asst_tUcre9CS6mNDuLm90VEahIZU'

thread = openai.beta.threads.create()

print(thread)

Thread(id='thread_cu8Z8Tv0LB4ZunmF2fpNLsBY', created_at=1717663158, metadata={}, object='thread', tool_resources=ToolResources(code_interpreter=None, file_search=None))


In [ ]:
message = openai.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="나는 서울 용산구에 있어, 뭐를 입어야 할까?"
)

run = openai.beta.threads.runs.create(
  thread_id=thread.id,  # 생성한 스레드 ID
  assistant_id=ASSISTANT_ID  # 적용할 Assistant ID
)
run_id=run.id

print(run)

In [13]:

run = openai.beta.threads.runs.retrieve(
  thread_id=thread.id,  # 생성한 스레드 ID
  run_id=run_id
)

if run.status == 'completed':
  messages = openai.beta.threads.messages.list(
    thread_id=thread.id
  )
  print(messages.data[0].content[0].text.value)
else:
  print(run.status)


requires_action


In [14]:
# Define the list to store tool outputs
tool_outputs = []
 
# Loop through each tool in the required action section
for tool in run.required_action.submit_tool_outputs.tool_calls:
  if tool.function.name == "weather_info":
    tool_outputs.append({
      "tool_call_id": tool.id,
      "output": "57"
    })
 
# Submit all tool outputs at once after collecting them in a list
if tool_outputs:
  try:
    run = openai.beta.threads.runs.submit_tool_outputs_and_poll(
      thread_id=thread.id,
      run_id=run.id,
      tool_outputs=tool_outputs
    )
  except Exception as e:
    print(e)
else:
  print("No tool outputs to submit.")
 
if run.status == 'completed':
  messages = openai.beta.threads.messages.list(
    thread_id=thread.id
  )
  print(messages.data[1].content[0].text.value)
  print(messages.data[0].content[0].text.value)
else:
  print(run.status)

나는 서울 용산구에 있어, 뭐를 입어야 할까?
서울 용산구의 오늘 날씨를 알려줄게.

어제와 비교해보면:
- 오늘은 어제보다 더 덥습니다.
- 오늘 기온이 30도 이상이라 엄청 덥습니다.

오늘 추천하는 옷차림:
- 가벼운 반팔 옷이나 시원한 옷을 입는 것이 좋습니다.
- 일교차가 크지 않지만, 낮 기온이 높아 더위를 많이 탈 수 있으니 주의하세요.

더운 날 건강 관리 잘하시고 시원하게 보내세요!
